In [1]:
#import tensorflow as tf
import numpy as np
import time
import pandas as pd
from os import listdir
from os.path import isfile, isdir, join
from PIL import Image
import cv2
import math
#import argparse
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('MNSIT_data',one_hot=True)

def rotate(image, angle, center=None, scale=1.0):
    # 獲取圖像尺寸
    (h, w) = image.shape[:2]
    # 若未指定旋轉中心，则將圖像中心設為旋轉中心
    if center is None:
        center = (w / 2, h / 2)
    # 執行旋轉
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    # 返回旋轉後的圖像
    return rotated

In [2]:
##將cvs裡面答案轉為陣列
df = pd.read_csv('validation.csv',header=None)  
df = df.T
df_temp = pd.get_dummies(df[0]) #One-hot編碼
y_r = np.array(df_temp)
##將驗證圖片讀檔，二值化並存成2維陣列200*(784)
img_set = []

#img_merge = np.zeros((20*28,10*28))#組合商品圖為一張，方便看

# 指定要列出所有檔案的目錄
mypath = "./validation"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
# 以迴圈處理
j=0
size = 28
for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.imread(fullpath,0)
        image = cv2.imread(fullpath)
        #img_merge[j%20*28:j%20*28+28,j//20*28: j//20*28+28]=im
        retval, im_bin = cv2.threshold(im,215, 255, cv2.THRESH_BINARY_INV)
        retval, labels,stats,centroids = cv2.connectedComponentsWithStats(im_bin)
        x,y,w,h,area=stats[1]
        cx,cy = centroids[1]

        if w<5:
            x=math.ceil(cx-3)
            w=w+4
        if h<5 :
            y=math.ceil(cy-3)
            h=h+6
        if w > h+1 and w >10 :
            #ro= True
            img = rotate(im,90)
            crop_img = img[x-1:x+w+1,y-1:y+h+1]
        else:
            crop_img = im[y:y+h,x:x+w]
        im_h ,im_w= crop_img.shape
        r = int(min(24/im_w,24/im_h))
        if r<=0: r=1
        if r>=4:r=2
        crop_img = cv2.resize(crop_img,(im_w*r,im_h*r),interpolation=cv2.INTER_AREA)#interpolation=cv2.INTER_NEAREST
        im_h ,im_w= crop_img.shape
        u = int((size-im_h)/2)
        v = int((size-im_w)/2)
        crop_img = cv2.copyMakeBorder(crop_img,u,size-im_h-u,v,size-im_w-v, cv2.BORDER_CONSTANT,value=[255,255,255])
        crop_img = cv2.erode(crop_img,(3,3), iterations=1)
        #cv2.imwrite('./Desktop/new/'+f,crop_img)
        
        img = np.array(crop_img,dtype=np.float32).reshape(28,28,1)
        img_set.append(1-img/255)
img_set= np.array(img_set)        

In [3]:
batch_size = 150

from keras.utils import np_utils
from keras.datasets import mnist
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential
import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from matplotlib.pyplot import imshow
import numpy as np


# Build the neural network! 
model = Sequential() 
# First convolutional layer with max pooling 
model.add(Conv2D(32, (5, 5), padding="same", input_shape=(28, 28, 1), activation="relu")) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
# Second convolutional layer with max pooling 
model.add(Conv2D(64, (5, 5), padding="same", activation="relu")) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
# Hidden layer with 500 nodes 
model.add(Flatten()) 
model.add(Dense(800, activation="relu")) 
# Output layer with 10 nodes (one for each possible letter/number we predict) 
model.add(Dense(10, activation="softmax")) 
# Ask Keras to build the TensorFlow model behind the scenes 
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 800)               2509600   
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [4]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
X_train = x_train.reshape(60000,28,28,1)/255
X_test = x_test.reshape(10000,28,28,1)/255

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 訓練

In [5]:
# Train the neural network 
model.fit(X_train[0:5000], y_train[0:5000], validation_data=(X_test[0:200], y_test[0:200]), batch_size=batch_size, epochs=25, verbose=1)

Train on 5000 samples, validate on 200 samples
Epoch 1/25
5000/5000 [==============================] - 5s 971us/step - loss: 0.8058 - accuracy: 0.7560 - val_loss: 0.2112 - val_accuracy: 0.9500
Epoch 2/25
5000/5000 [==============================] - 5s 926us/step - loss: 0.1856 - accuracy: 0.9452 - val_loss: 0.0807 - val_accuracy: 0.9850
Epoch 3/25
5000/5000 [==============================] - 5s 984us/step - loss: 0.1036 - accuracy: 0.9708 - val_loss: 0.0675 - val_accuracy: 0.9800
Epoch 4/25
5000/5000 [==============================] - 5s 1ms/step - loss: 0.0673 - accuracy: 0.9806 - val_loss: 0.0307 - val_accuracy: 0.9900
Epoch 5/25
5000/5000 [==============================] - 5s 935us/step - loss: 0.0490 - accuracy: 0.9868 - val_loss: 0.0257 - val_accuracy: 0.9950
Epoch 6/25
5000/5000 [==============================] - 5s 954us/step - loss: 0.0314 - accuracy: 0.9924 - val_loss: 0.0189 - val_accuracy: 0.9950
Epoch 7/25
5000/5000 [==============================] - 5s 945us/step - loss: 0

## 測試

In [6]:
result = model.predict_classes(img_set)
print(result)
score = 0
for pre,ans in zip(list(result),list(np.array(df[0]))): 
    if pre ==ans:
        score=score+1
    else:pass
print("Testing Accuracy:",score/200)

[7 7 8 8 1 1 3 4 4 5 5 6 6 3 2 2 1 1 0 0 7 0 8 8 2 3 3 8 4 5 5 6 6 0 3 3 1
 2 4 3 0 0 1 1 7 2 3 4 6 5 5 4 4 3 9 9 8 1 8 0 9 6 3 6 7 9 5 2 2 1 4 1 1 9
 1 0 4 7 7 3 0 6 3 4 6 2 8 5 2 9 9 1 8 4 1 1 7 7 6 0 7 7 5 0 4 3 4 4 4 4 4
 4 0 2 1 6 1 1 0 1 9 4 3 0 4 5 7 0 3 3 2 4 4 8 5 6 6 9 9 8 8 8 8 8 8 8 8 8
 8 7 7 8 8 6 6 5 5 4 4 3 3 2 2 1 1 1 0 9 1 6 6 8 4 9 9 2 1 5 8 9 4 1 3 8 7
 7 3 3 4 3 2 3 6 2 3 5 2 3 3 6]
Testing Accuracy: 0.68
